### This file explores possibilities of turning ReaSCAN to a program synthesis abstract reasoning corpus.
This is for future-looking purpose only. Personally, this is for solving the ARC challenge (Chollet 2019).

In [1]:
# We turn this navigation task into an abstract reasoning challenge.

# Steps:
# We still start with a command that can help us to identify a referent target.
# Then, we generate a set of them. 
# Next, we apply some changes to the referent target.
# Finally, generate the output, which will be another target situation.
# The task now becomes: from the input and output situation, guess the test output when test input is provided.

In [2]:
from collections import namedtuple, OrderedDict
import itertools
import os
import numpy as np
from typing import Tuple
from typing import List
from typing import Dict
import random
from itertools import product
import copy
import re
import random

from utils import one_hot
from utils import generate_possible_object_names
from utils import numpy_array_to_image

from vocabulary import *
from object_vocabulary import *
from world import *
from grammer import *
from simulator import *
from relation_graph import *

from networkx.algorithms import isomorphism
from networkx import DiGraph
from networkx import line_graph

import math

In [25]:
# test out the vocabulary
intransitive_verbs = ["walk"]
transitive_verbs = ["push", "pull"]
adverbs = ["while zigzagging", "while spinning", "cautiously", "hesitantly"]
nouns = ["circle", "cylinder", "square", "box"]
color_adjectives = ["red", "blue", "green", "yellow"]
size_adjectives = ["big", "small"]
relative_pronouns = ["that is"]
relation_clauses = ["in the same row as", 
                    "in the same column as", 
                    "in the same color as", 
                    "in the same shape as", 
                    "in the same size as",
                    "inside of"]
vocabulary = Vocabulary.initialize(intransitive_verbs=intransitive_verbs,
                                   transitive_verbs=transitive_verbs, adverbs=adverbs, nouns=nouns,
                                   color_adjectives=color_adjectives,
                                   size_adjectives=size_adjectives, 
                                   relative_pronouns=relative_pronouns, 
                                   relation_clauses=relation_clauses)
# test out the object vocab
min_object_size = 1
max_object_size = 4
object_vocabulary = ObjectVocabulary(shapes=vocabulary.get_semantic_shapes(),
                                     colors=vocabulary.get_semantic_colors(),
                                     min_size=min_object_size, max_size=max_object_size)
# object_vocabulary.generate_objects()

# Generating all the core command structs.
grammer = Grammer(vocabulary)

In [31]:
def generate_abstract_task_input_situation(
    test_struct,
    simulator,
    n_examples=4
):

    obj_pattern_map = test_struct["obj_pattern_map"]
    rel_map = test_struct["rel_map"]
    obj_map = test_struct["obj_map"]
    grammer_pattern = test_struct["grammer_pattern"]
    verb = test_struct["verb"]
    adverb = test_struct["adverb"]

    test_unique_find = 0
    n_try_max = 500
    
    input_situations = []

    for _ in range(n_examples):
        for i in range(n_try_max):
            sampled_world = simulator.sample_situations_from_grounded_grammer(
                    copy.deepcopy(grammer_pattern), 
                    copy.deepcopy(obj_pattern_map), 
                    copy.deepcopy(rel_map), 
                    copy.deepcopy(obj_map),
                    is_plot=False,
                    include_relation_distractor=True, 
                    include_attribute_distractor=False, 
                    include_isomorphism_distractor=False, 
                    include_random_distractor=True,
                    full_relation_probability=1.0, # 0.5 seems to work as well!
                    debug=False
                )

            assert len(sampled_world['obj_map']) == len(simulator._world.get_current_situation().to_representation()["placed_objects"])

            graph = ReaSCANGraph(
                objects=sampled_world["obj_map"], 
                object_patterns=sampled_world["obj_pattern_map"], 
                vocabulary=vocabulary,
                positions=sampled_world["pos_map"], 
                referred_object=sampled_world["referred_obj"],
                debug=False
            )

            pattern_graph = ReaSCANGraph(
                objects=obj_map, 
                object_patterns=None,
                vocabulary=vocabulary,
                relations=rel_map, 
                referred_object='$OBJ_0', 
                debug=False
            )

            potential_referent_target = graph.find_referred_object_super_fast(
                pattern_graph, referred_object='$OBJ_0', 
                debug=False
            )

            if len(potential_referent_target) == 1:
                test_unique_find += 1

                # Form the command with grounded determiners!
                obj_determiner_map = graph.find_determiners(
                    pattern_graph, 
                    referred_object='$OBJ_0', 
                    debug=False,
                )
                command_str = grammer.repre_str_command(
                    grammer_pattern, rel_map, obj_map, 
                    obj_determiner_map, 
                    verb,
                    adverb,
                )
                input_situations += [{
                    "obj_map": sampled_world["obj_map"], 
                    "pos_map": sampled_world["pos_map"]
                }]
                break
    print(f"command = {command_str}")
    return input_situations

def generate_abstract_task_output_situtation(
    input_situations,
    operations="DRAW"
):
    output_situtations = copy.deepcopy(input_situations)
    for i in range(0, len(output_situtations)):
        referred_object = "$OBJ_0"
        if operations == "DRAW":
            output_situtations[i]['obj_map']["$OBJ_0"] = Object(
                size=output_situtations[i]['obj_map']["$OBJ_0"].size,
                color="red",
                shape=output_situtations[i]['obj_map']["$OBJ_0"].shape,
            )
        elif operations == "MOVE":
            pass # need some other ways.
#             target_position = (
#                 output_situtations[i]['pos_map']["$OBJ_0"].row,
#                 output_situtations[i]['pos_map']["$OBJ_0"].column+1,
#             )
#             positions = set([])
#             for k, v in output_situtations[i]['pos_map'].items():
#                 positions.add((v.row, v.column))
#             if target_position not in positions:
#                 output_situtations[i]['pos_map']["$OBJ_0"] = Position(
#                     column=target_position[1],
#                     row=target_position[0],
#                 )
        elif operations == "ENLARGE":
            output_situtations[i]['obj_map']["$OBJ_0"] = Object(
                size=min(output_situtations[i]['obj_map']["$OBJ_0"].size+1, 4),
                color=output_situtations[i]['obj_map']["$OBJ_0"].color,
                shape=output_situtations[i]['obj_map']["$OBJ_0"].shape,
            )
        elif operations == "CHANGE_SHAPE":
            output_situtations[i]['obj_map']["$OBJ_0"] = Object(
                size=1,
                color=output_situtations[i]['obj_map']["$OBJ_0"].color,
                shape="box",
            )
        elif operations == "DRAW_ADJACENT":
            target_position = (
                output_situtations[i]['pos_map']["$OBJ_0"].row,
                output_situtations[i]['pos_map']["$OBJ_0"].column+1,
            )
            positions = set([])
            for k, v in output_situtations[i]['pos_map'].items():
                positions.add((v.row, v.column))
            if target_position not in positions:
                new_obj_str = "$OBJ_NEW"
                output_situtations[i]['obj_map'][new_obj_str] = Object(
                    size=4,
                    color="blue",
                    shape="square",
                )
                output_situtations[i]['pos_map'][new_obj_str] = Position(
                    column=target_position[1],
                    row=target_position[0],
                )
    return output_situtations

def generate_task(
    input_situations, 
    output_situations,
    world,
    img_format="grid" # ideally, we should support non-grid black bg color reasoning format as ARC
):
    tasks = []
    
    if img_format == "grid":
        for i in range(0, len(input_situations)):
            # input image
            world.clear_situation()
            for obj_idx, obj in input_situations[i]["obj_map"].items():
                world.place_object(
                    obj, 
                    position=input_situations[i]["pos_map"][obj_idx]
                )
            in_array = world.render_simple(
                array_only=True, include_agent=False
            )

            # input image
            world.clear_situation()
            for obj_idx, obj in output_situations[i]["obj_map"].items():
                world.place_object(
                    obj, 
                    position=output_situations[i]["pos_map"][obj_idx]
                )
            out_array = world.render_simple(
                array_only=True, include_agent=False
            )

            tasks += [{
                "input": in_array,
                "output": out_array,
            }]

    return tasks

def visualize_task(
    task,
    img_format="grid",
    save_file=None,
):
    # test out the world
    import matplotlib as plt
    import matplotlib.pyplot as plt
    from matplotlib import gridspec

    N = 7
    cols = 2
    rows = int(math.ceil(N / cols))

    gs = gridspec.GridSpec(rows, cols)
    fig = plt.figure(figsize=(8,16))

    # a set of plots for distractor type dist
    ax = fig.add_subplot(gs[0])
    ax.imshow(task[0]["input"])
    plt.xticks([])
    plt.yticks([])
    ax = fig.add_subplot(gs[1])
    ax.imshow(task[0]["output"])
    plt.xticks([])
    plt.yticks([])
    
    ax = fig.add_subplot(gs[2])
    ax.imshow(task[1]["input"])
    plt.xticks([])
    plt.yticks([])
    ax = fig.add_subplot(gs[3])
    ax.imshow(task[1]["output"])
    plt.xticks([])
    plt.yticks([])
    
    ax = fig.add_subplot(gs[4])
    ax.imshow(task[2]["input"])
    plt.xticks([])
    plt.yticks([])
    ax = fig.add_subplot(gs[5])
    ax.imshow(task[2]["output"])
    plt.xticks([])
    plt.yticks([])
    
    ax = fig.add_subplot(gs[6])
    ax.imshow(task[3]["input"])
    plt.xticks([])
    plt.yticks([])
    
    if save_file is not None:
        fig.savefig(
            save_file,
            dpi=1000, bbox_inches='tight'
        )
        plt.close(fig)

In [28]:
simulator = Simulator(
    object_vocabulary, vocabulary, 
    grid_size=6, 
    n_object_max=6,
)

world = World(grid_size=6, colors=vocabulary.get_semantic_colors(),
              object_vocabulary=object_vocabulary,
              shapes=vocabulary.get_semantic_shapes(),
              save_directory="./tmp/"
)

In [35]:
test_struct = {'obj_pattern_map': {'$OBJ_0': '$ABS_SHAPE',
  '$OBJ_1': '$COLOR $SHAPE'},
 'rel_map': OrderedDict([(('$OBJ_0', '$OBJ_1'), '$SAME_ROW')]),
 'obj_map': {'$OBJ_0': 'object',
  '$OBJ_1': 'red circle'},
 'grammer_pattern': '$OBJ_0 ^ $OBJ_1',
 'adverb': '',
 'verb': 'SELECT'}

input_situations = generate_abstract_task_input_situation(
    test_struct,
    simulator,
    n_examples=4
)

output_situations = generate_abstract_task_output_situtation(
    input_situations,
    operations="DRAW"
)

task = generate_task(
    input_situations, 
    output_situations,
    world,
    img_format="grid"
)

command = SELECT to the object that is in the same row as the red circle


In [36]:
visualize_task(
    task,
    save_file="../../data-files-updated/Analysis-Result/arc_example.png"
)